In [6]:
import pandas as pd
from pybaseball import statcast
pd.set_option('display.max_columns', 500)

In [2]:
def fetch_statcast_data(start_date, end_date):
    # Fetch data from Statcast
    data = statcast(start_date, end_date)
    return data

In [34]:
def process_data(data):
    # Filter necessary columns
    data = data[['game_date', 'batter', 'pitcher', 'pitch_type', 'events', 'at_bat_number', 'pitch_number']]
    
    # Drop rows where pitch_type is NaN (happens occasionally in real datasets)
    data = data.dropna(subset=['pitch_type'])

    # Group data by game, batter, pitcher, and at_bat_number, then sort by pitch_number
    # Use apply to ensure the group maintains DataFrame format after sorting
    grouped = data.groupby(['game_date', 'batter', 'pitcher', 'at_bat_number']).apply(lambda x: x.sort_values('pitch_number')).reset_index(drop=True)

    # We can't directly iterate with multiple unpacks if using apply without specifying 'group_keys=False' in the groupby
    # However, by resetting the index, we revert to a single DataFrame and thus lose the direct tuple unpacking ability
    # Create a DataFrame to store the sequence, pitcher ID, and at-bat outcome
    at_bats = []
    for _, group in grouped.groupby(['game_date', 'batter', 'pitcher', 'at_bat_number']):
        sequence = ','.join(group['pitch_type'].tolist())
        outcome = group['events'].iloc[-1] if not pd.isna(group['events'].iloc[-1]) else 'No event'
        pitcher_id = group['pitcher'].iloc[0]
        at_bats.append([sequence, pitcher_id, outcome])

    # Convert list to DataFrame
    result_df = pd.DataFrame(at_bats, columns=['Pitch Sequence', 'Pitcher ID', 'At-Bat Outcome'])
    return result_df


In [22]:
def consolidate_pitch_types(data):
    # Mapping dictionary for pitch types
    pitch_type_map = {
        'CH': 'CH',  # Changeup
        'CU': 'CB',  # Curveball
        'FC': 'FC',  # Cutter
        'EP': 'CH',  # Eephus
        'FO': 'CH',  # Forkball
        'FF': 'FF',  # Four-Seam Fastball
        'KN': 'KN',  # Knuckleball
        'KC': 'CB',  # Knuckle-curve
        'SC': 'CH',  # Screwball
        'SI': 'SI',  # Sinker
        'SL': 'SL',  # Slider
        'SV': 'CB',  # Slurve
        'FS': 'FS',  # Splitter
        'ST': 'ST',  # Sweeper
        'FA': 'FF',  # Fastball
        'PO': 'PO',  # Pitch out
        'CS': 'CB'   # Slow Curveball
    }
    
    # Apply the mapping to the pitch_type column
    data['pitch_type'] = data['pitch_type'].map(pitch_type_map).fillna(data['pitch_type'])
    return data

In [23]:
start_date = '2023-04-01'
end_date = '2023-04-30'
data = fetch_statcast_data(start_date, end_date)

This is a large query, it may take a moment to complete


100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [00:10<00:00,  2.89it/s]


In [37]:
data.columns

Index(['pitch_type', 'game_date', 'release_speed', 'release_pos_x',
       'release_pos_z', 'player_name', 'batter', 'pitcher', 'events',
       'description', 'spin_dir', 'spin_rate_deprecated',
       'break_angle_deprecated', 'break_length_deprecated', 'zone', 'des',
       'game_type', 'stand', 'p_throws', 'home_team', 'away_team', 'type',
       'hit_location', 'bb_type', 'balls', 'strikes', 'game_year', 'pfx_x',
       'pfx_z', 'plate_x', 'plate_z', 'on_3b', 'on_2b', 'on_1b',
       'outs_when_up', 'inning', 'inning_topbot', 'hc_x', 'hc_y',
       'tfs_deprecated', 'tfs_zulu_deprecated', 'fielder_2', 'umpire', 'sv_id',
       'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'sz_top', 'sz_bot',
       'hit_distance_sc', 'launch_speed', 'launch_angle', 'effective_speed',
       'release_spin_rate', 'release_extension', 'game_pk', 'pitcher.1',
       'fielder_2.1', 'fielder_3', 'fielder_4', 'fielder_5', 'fielder_6',
       'fielder_7', 'fielder_8', 'fielder_9', 'release_pos_y',
       'estima

In [25]:
data_pt = consolidate_pitch_types(data)

In [43]:
data[['game_date', 'batter', 'pitcher', 'pitch_type', 'events', 'at_bat_number','pitch_number','player_name']].head(11)

,game_date,batter,pitcher,pitch_type,events,at_bat_number,pitch_number,player_name
2430,2023-04-30,665926,657514,SI,field_out,72,5,"Bernardino, Brennan"
2537,2023-04-30,665926,657514,SI,NaN,72,4,"Bernardino, Brennan"
2630,2023-04-30,665926,657514,SI,NaN,72,3,"Bernardino, Brennan"
2669,2023-04-30,665926,657514,SI,NaN,72,2,"Bernardino, Brennan"
2808,2023-04-30,665926,657514,SI,NaN,72,1,"Bernardino, Brennan"
2910,2023-04-30,660757,657514,SI,field_out,71,6,"Bernardino, Brennan"
2997,2023-04-30,660757,657514,SI,NaN,71,5,"Bernardino, Brennan"
3149,2023-04-30,660757,657514,SI,NaN,71,4,"Bernardino, Brennan"
3287,2023-04-30,660757,657514,SI,NaN,71,3,"Bernardino, Brennan"
3413,2023-04-30,660757,657514,SI,NaN,71,2,"Bernardino, Brennan"


In [44]:
data_pt.pitch_type.value_counts(normalize=True)

FF    0.322067
SL    0.167108
SI    0.147506
CH    0.107686
CB    0.096394
FC    0.082580
ST    0.054942
FS    0.021667
PO    0.000050
Name: pitch_type, dtype: float64

In [35]:
result_df = process_data(data_pt)
print(result_df.head())

         Pitch Sequence  Pitcher ID             At-Bat Outcome
0                    SI      621107                field_error
1  SI,CB,FC,SI,CB,SI,FF      621107                     single
2     ST,ST,SI,SI,ST,ST      676534                       walk
3        SI,ST,SI,ST,SI      687330  grounded_into_double_play
4     FF,FF,FF,SL,FF,SL      477132                  strikeout


In [47]:
data_pt[data_pt['pitcher']==621107]['player_name']

482     Eflin, Zach
507     Eflin, Zach
526     Eflin, Zach
555     Eflin, Zach
563     Eflin, Zach
           ...     
3860    Eflin, Zach
3993    Eflin, Zach
4188    Eflin, Zach
4258    Eflin, Zach
4344    Eflin, Zach
Name: player_name, Length: 297, dtype: object

In [36]:
result_df.shape

(30671, 3)

In [52]:
result_df.head()

,Pitch Sequence,Pitcher ID,At-Bat Outcome
0,SI,621107,field_error
1,"SI,CB,FC,SI,CB,SI,FF",621107,single
2,"ST,ST,SI,SI,ST,ST",676534,walk
3,"SI,ST,SI,ST,SI",687330,grounded_into_double_play
4,"FF,FF,FF,SL,FF,SL",477132,strikeout


In [51]:
result_df.to_csv('../../data/sequence_data_opt.csv', index=False)